# Assignment 2

## 31994695 June Jin


## Part A (20%)

### Task 1. MongoDB Data Model (5%)

First, to create a new database with the desired data, we need to read the CSV files using pandas. Then, we'll each examine our data to determine which attributes would be most useful for comparing the two datasets. My idea was storing the hotspot_historic data based on the observed dates in the climate_historic data because I think it would be easier to manage the data that way. Therefore, I proposed using an inner join to align the data based on the dates.

Although there is only one set of climate data, the number of hotspot datasets within it can range from zero to multiple. Therefore, I create a list of hotspots for each date corresponding to the climate data. For each date in hotspot_historic, I check if it matches the date in the climate data and then add climate data - hotspot[] accordingly.

For example, I m assuming this format below:
 ```json
{'GHI_w/m2': 170,
  'air_temperature_celcius': 22,
  'date': Timestamp('2023-04-03 00:00:00'),
  'hotspots': [],
  'max_wind_speed': 11.1,
  'precipitation': '0.00I',
  'relative_humidity': 62.0,
  'station': 948701,
  'windspeed_knots': 7.9},
 {'GHI_w/m2': 159,
  'air_temperature_celcius': 20,
  'date': Timestamp('2023-05-03 00:00:00'),
  'hotspots': [],
  'max_wind_speed': 15.9,
  'precipitation': '0.00I',
  'relative_humidity': 58.8,
  'station': 948701,
  'windspeed_knots': 11.5},
 {'GHI_w/m2': 167,
  'air_temperature_celcius': 20,
  'date': Timestamp('2023-06-03 00:00:00'),
  'hotspots': [{'confidence': 87,
                'datetime': Timestamp('2023-03-06 05:06:30'),
                'latitude': -34.3795,
                'longitude': 141.6331,
                'surface_temperature_celcius': 62},
               {'confidence': 85,
                'datetime': Timestamp('2023-03-06 05:06:20'),
                'latitude': -34.3735,
                'longitude': 141.6604,
                'surface_temperature_celcius': 59}],
  'max_wind_speed': 21.0,
  'precipitation': '0.00I',
  'relative_humidity': 52.9,
  'station': 948701,
  'windspeed_knots': 17.0}

```

The reason I considered date to be the most suitable attribute is that, for predicting fires in Victorian cities, data based on time and date would be most conducive to fire prediction.

### Task 2. Querying MongoDB using PyMongo (15%)

#### Task 2 - 1

In [80]:
import pandas as pd
import pymongo
from pymongo import MongoClient
from pprint import pprint
from datetime import datetime
import json
import re
import csv

In [81]:
# "climate_historic.csv" file to data frame with pandas
climate_historic_df = pd.read_csv("./climate_historic.csv")

# make datetime format
climate_historic_df['date'] = pd.to_datetime(climate_historic_df['date'])

# make hotspot historic to data frame with pandas
hotspot_historic_df = pd.read_csv("./hotspot_historic.csv")

# make datetime format
hotspot_historic_df['date'] = pd.to_datetime(hotspot_historic_df['date'])
hotspot_historic_df['datetime'] = pd.to_datetime(hotspot_historic_df['datetime'])

# trying
#hotspot_historic_df = pd.read_csv("./hotspot_historic.csv", parse_dates=['datetime'])


/tmp/ipykernel_4112/536444155.py:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  climate_historic_df['date'] = pd.to_datetime(climate_historic_df['date'])
/tmp/ipykernel_4112/536444155.py:11: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  hotspot_historic_df['date'] = pd.to_datetime(hotspot_historic_df['date'])


In [82]:
# 1. climate_historic make this data to dict
climate_historic_list = climate_historic_df.to_dict('records')


In [83]:
for climate_info in climate_historic_list:
    climate_info['hotspots'] = []
    for hotspot_info in hotspot_historic_list:
        if climate_info['date'] == hotspot_info['date']:
            climate_info['hotspots'].append({
                'latitude': hotspot_info['latitude'],
                'longitude': hotspot_info['longitude'],
                'datetime': hotspot_info['datetime'],
                'confidence': hotspot_info['confidence'],
                'surface_temperature_celcius': hotspot_info['surface_temperature_celcius']
            })


In [84]:
# use the MongoDB URL format

# campus
client = MongoClient('mongodb://10.192.1.37:27017/')

# home
#client = MongoClient('mongodb://192.168.1.101:27017/')

# list of database before we delete
result = client.list_database_names()
#print(result)

# drop current used database
client.drop_database('fit3182_assignment1_db')

# make database
db = client.fit3182_assignment1_db

# add new collection into new database
collection = db.merged_history

# insert data to collection
collection.insert_many(climate_historic_list)



#### Task 2 - 2 a

In [85]:
# search the data via query
result = collection.find({"date": {
                         '$gte': datetime(2023, 12, 12),
                         '$lt' : datetime(2023, 12, 13)}})

# output
for document in result:
    pprint(document)



{'GHI_w/m2': 156,
 '_id': ObjectId('664f0a66ab9e5e59a44691cb'),
 'air_temperature_celcius': 19,
 'date': datetime.datetime(2023, 12, 12, 0, 0),
 'hotspots': [{'confidence': 53,
               'datetime': datetime.datetime(2023, 12, 12, 0, 45, 38),
               'latitude': -37.903,
               'longitude': 145.25,
               'surface_temperature_celcius': 44}],
 'max_wind_speed': 12.0,
 'precipitation': '0.00I',
 'relative_humidity': 55.3,
 'station': 948702,
 'windspeed_knots': 6.2}


Therefore, we could find that there is 1 climate data that match with date.

#### Task 2-2 b

In [86]:
result = collection.find(
    {"hotspots.surface_temperature_celcius": {"$gte": 65, "$lte": 100}},
    {
        "hotspots.latitude": 1,
        "hotspots.longitude": 1,
        "hotspots.surface_temperature_celcius": 1,
        "hotspots.confidence": 1,
        "_id": 0  
    }
)

# output
for document in result:
    pprint(document)


{'hotspots': [{'confidence': 100,
               'latitude': -37.223,
               'longitude': 147.9431,
               'surface_temperature_celcius': 105},
              {'confidence': 100,
               'latitude': -37.2252,
               'longitude': 147.9363,
               'surface_temperature_celcius': 109},
              {'confidence': 94,
               'latitude': -37.2284,
               'longitude': 147.9187,
               'surface_temperature_celcius': 73},
              {'confidence': 97,
               'latitude': -37.6572,
               'longitude': 142.0703,
               'surface_temperature_celcius': 80},
              {'confidence': 81,
               'latitude': -37.6592,
               'longitude': 142.058,
               'surface_temperature_celcius': 55},
              {'confidence': 55,
               'latitude': -36.2544,
               'longitude': 148.0353,
               'surface_temperature_celcius': 42},
              {'confidence': 54,
           

#### Task 2-2 c

In [87]:
from datetime import datetime

result = collection.aggregate([
    {"$unwind": "$hotspots"},  # 'hotspots' 배열 풀기
    {"$match": {
        "date": {
            '$gte': datetime(2023, 12, 15),
            '$lt': datetime(2023, 12, 17)
        }
    }},
    {
        "$project": {
            "date": 1,
            "hotspots.surface_temperature_celcius": 1,
            "air_temperature_celcius": 1,
            "relative_humidity": 1,
            "max_wind_speed": 1,
            "_id": 0
        }
    }
])

# output
for document in result:
    pprint(document)


{'air_temperature_celcius': 18,
 'date': datetime.datetime(2023, 12, 15, 0, 0),
 'hotspots': {'surface_temperature_celcius': 42},
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'air_temperature_celcius': 18,
 'date': datetime.datetime(2023, 12, 15, 0, 0),
 'hotspots': {'surface_temperature_celcius': 36},
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'air_temperature_celcius': 18,
 'date': datetime.datetime(2023, 12, 15, 0, 0),
 'hotspots': {'surface_temperature_celcius': 38},
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'air_temperature_celcius': 18,
 'date': datetime.datetime(2023, 12, 15, 0, 0),
 'hotspots': {'surface_temperature_celcius': 40},
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'air_temperature_celcius': 18,
 'date': datetime.datetime(2023, 12, 16, 0, 0),
 'hotspots': {'surface_temperature_celcius': 43},
 'max_wind_speed': 13.0,
 'relative_humidity': 53.7}
{'air_temperature_celcius': 18,
 'date': datetime.datetime(2023, 12, 16, 0, 0),
 'hot

#### Task 2-2 d

In [88]:
result = collection.aggregate([
    {"$unwind": "$hotspots"},  
    {"$match": {"hotspots.confidence": {"$gte": 80, "$lte": 100}}},
    {
        "$project": {
            "hotspots.datetime": 1,
            "air_temperature_celcius": 1,
            "hotspots.surface_temperature_celcius": 1,
            "hotspots.confidence": 1,
            "_id": 0  
        }
    }
])

# output
for document in result:
    pprint(document)


{'air_temperature_celcius': 20,
 'hotspots': {'confidence': 87,
              'datetime': datetime.datetime(2023, 3, 6, 5, 6, 30),
              'surface_temperature_celcius': 62}}
{'air_temperature_celcius': 20,
 'hotspots': {'confidence': 85,
              'datetime': datetime.datetime(2023, 3, 6, 5, 6, 20),
              'surface_temperature_celcius': 59}}
{'air_temperature_celcius': 19,
 'hotspots': {'confidence': 88,
              'datetime': datetime.datetime(2023, 3, 7, 4, 16, 10),
              'surface_temperature_celcius': 64}}
{'air_temperature_celcius': 23,
 'hotspots': {'confidence': 86,
              'datetime': datetime.datetime(2023, 3, 9, 13, 23, 40),
              'surface_temperature_celcius': 41}}
{'air_temperature_celcius': 19,
 'hotspots': {'confidence': 100,
              'datetime': datetime.datetime(2023, 3, 10, 4, 48, 40),
              'surface_temperature_celcius': 105}}
{'air_temperature_celcius': 19,
 'hotspots': {'confidence': 100,
              'datetime

#### Task 2-2 e date sort

In [89]:
result = collection.aggregate([
    {"$unwind": "$hotspots"}, 
    {"$sort": {"hotspots.surface_temperature_celcius": pymongo.DESCENDING}},  
    {"$limit": 10}  
])

# output
for document in result:
    pprint(document)


{'GHI_w/m2': 122,
 '_id': ObjectId('664f0a66ab9e5e59a44690dd'),
 'air_temperature_celcius': 15,
 'date': datetime.datetime(2023, 4, 18, 0, 0),
 'hotspots': {'confidence': 100,
              'datetime': datetime.datetime(2023, 4, 18, 4, 52),
              'latitude': -38.1665,
              'longitude': 143.062,
              'surface_temperature_celcius': 124},
 'max_wind_speed': 9.9,
 'precipitation': '0.00I',
 'relative_humidity': 56.1,
 'station': 948701,
 'windspeed_knots': 5.1}
{'GHI_w/m2': 140,
 '_id': ObjectId('664f0a66ab9e5e59a44690cf'),
 'air_temperature_celcius': 16,
 'date': datetime.datetime(2023, 4, 4, 0, 0),
 'hotspots': {'confidence': 100,
              'datetime': datetime.datetime(2023, 4, 4, 4, 32, 50),
              'latitude': -36.343,
              'longitude': 142.1986,
              'surface_temperature_celcius': 123},
 'max_wind_speed': 12.0,
 'precipitation': '0.00I',
 'relative_humidity': 47.5,
 'station': 948701,
 'windspeed_knots': 5.4}
{'GHI_w/m2': 121,
 '_

#### Task 2-2 f date sort

Find the number of fires each day. 
Required to only display the total number of fires and the date in the output.

- output the days which have no hotspots and return a value of 0 alongside the days with hotspots


In [90]:
result = collection.aggregate([
    {"$unwind": {
        'path': '$hotspots',
        "preserveNullAndEmptyArrays": True
    }},
    {"$group": {
        "_id": "$date",
        "count": {"$sum": {"$cond": [{"$ifNull": ["$hotspots", False]}, 1, 0]}}
    }},
    {"$sort": {"_id": 1}}
])

# output
for document in result:
    pprint(document)


{'_id': datetime.datetime(2022, 12, 31, 0, 0), 'count': 0}
{'_id': datetime.datetime(2023, 1, 2, 0, 0), 'count': 0}
{'_id': datetime.datetime(2023, 1, 3, 0, 0), 'count': 0}
{'_id': datetime.datetime(2023, 1, 4, 0, 0), 'count': 7}
{'_id': datetime.datetime(2023, 1, 5, 0, 0), 'count': 20}
{'_id': datetime.datetime(2023, 1, 6, 0, 0), 'count': 2}
{'_id': datetime.datetime(2023, 1, 7, 0, 0), 'count': 4}
{'_id': datetime.datetime(2023, 1, 8, 0, 0), 'count': 2}
{'_id': datetime.datetime(2023, 1, 9, 0, 0), 'count': 0}
{'_id': datetime.datetime(2023, 1, 10, 0, 0), 'count': 8}
{'_id': datetime.datetime(2023, 1, 11, 0, 0), 'count': 0}
{'_id': datetime.datetime(2023, 1, 12, 0, 0), 'count': 0}
{'_id': datetime.datetime(2023, 1, 13, 0, 0), 'count': 0}
{'_id': datetime.datetime(2023, 1, 14, 0, 0), 'count': 0}
{'_id': datetime.datetime(2023, 1, 15, 0, 0), 'count': 0}
{'_id': datetime.datetime(2023, 1, 16, 0, 0), 'count': 0}
{'_id': datetime.datetime(2023, 1, 17, 0, 0), 'count': 0}
{'_id': datetime.dat

#### Task 2-2 g

In [91]:
result = collection.aggregate([
    {"$unwind": "$hotspots"},  
    {"$match": {"hotspots.confidence": {"$lt": 70}}},  
    {"$sort": {"date": 1}},
    {"$project":{"_id" : 0, "hotspots" : 1}}
])

# output
for document in result:
    pprint(document)

{'hotspots': {'confidence': 58,
              'datetime': datetime.datetime(2023, 4, 1, 4, 2, 40),
              'latitude': -36.2044,
              'longitude': 145.7706,
              'surface_temperature_celcius': 51}}
{'hotspots': {'confidence': 57,
              'datetime': datetime.datetime(2023, 4, 1, 4, 2, 40),
              'latitude': -36.5245,
              'longitude': 143.1556,
              'surface_temperature_celcius': 43}}
{'hotspots': {'confidence': 50,
              'datetime': datetime.datetime(2023, 4, 1, 4, 2, 40),
              'latitude': -36.1002,
              'longitude': 147.6494,
              'surface_temperature_celcius': 39}}
{'hotspots': {'confidence': 55,
              'datetime': datetime.datetime(2023, 5, 1, 4, 16),
              'latitude': -36.2929,
              'longitude': 142.9282,
              'surface_temperature_celcius': 39}}
{'hotspots': {'confidence': 63,
              'datetime': datetime.datetime(2023, 5, 1, 4, 15, 40),
              '

#### Task 2-2 h

Find the average surface temperature (°C) for each day. You are required to
only display average surface temperature (°C) and the date in the output.

- Include N/A or None for the days which have zero hotspots.

In [92]:
results = collection.aggregate([
    {"$unwind": {
        'path': '$hotspots',
        "preserveNullAndEmptyArrays": True
    }},  
    {
        "$group": {
            "_id": "$date",
            "avg": {"$avg": "$hotspots.surface_temperature_celcius"}  
        }
    },
    {"$sort": {"_id": 1}}
    
    ]
    )

# output
for document in results:
    pprint(document)


{'_id': datetime.datetime(2022, 12, 31, 0, 0), 'avg': None}
{'_id': datetime.datetime(2023, 1, 2, 0, 0), 'avg': None}
{'_id': datetime.datetime(2023, 1, 3, 0, 0), 'avg': None}
{'_id': datetime.datetime(2023, 1, 4, 0, 0), 'avg': 46.714285714285715}
{'_id': datetime.datetime(2023, 1, 5, 0, 0), 'avg': 68.4}
{'_id': datetime.datetime(2023, 1, 6, 0, 0), 'avg': 54.0}
{'_id': datetime.datetime(2023, 1, 7, 0, 0), 'avg': 30.5}
{'_id': datetime.datetime(2023, 1, 8, 0, 0), 'avg': 58.0}
{'_id': datetime.datetime(2023, 1, 9, 0, 0), 'avg': None}
{'_id': datetime.datetime(2023, 1, 10, 0, 0), 'avg': 48.25}
{'_id': datetime.datetime(2023, 1, 11, 0, 0), 'avg': None}
{'_id': datetime.datetime(2023, 1, 12, 0, 0), 'avg': None}
{'_id': datetime.datetime(2023, 1, 13, 0, 0), 'avg': None}
{'_id': datetime.datetime(2023, 1, 14, 0, 0), 'avg': None}
{'_id': datetime.datetime(2023, 1, 15, 0, 0), 'avg': None}
{'_id': datetime.datetime(2023, 1, 16, 0, 0), 'avg': None}
{'_id': datetime.datetime(2023, 1, 17, 0, 0), 'a

#### Task 2-2 i

In [93]:
result = collection.find().sort("GHI_w/m2", pymongo.ASCENDING).limit(10)

for i in result:
    pprint(i)

{'GHI_w/m2': 47,
 '_id': ObjectId('664f0a66ab9e5e59a4469147'),
 'air_temperature_celcius': 5,
 'date': datetime.datetime(2023, 2, 8, 0, 0),
 'hotspots': [{'confidence': 94,
               'datetime': datetime.datetime(2023, 8, 2, 3, 45, 40),
               'latitude': -37.4796,
               'longitude': 141.9403,
               'surface_temperature_celcius': 87},
              {'confidence': 54,
               'datetime': datetime.datetime(2023, 8, 2, 3, 45),
               'latitude': -37.491,
               'longitude': 141.936,
               'surface_temperature_celcius': 40}],
 'max_wind_speed': 5.1,
 'precipitation': '0.00I',
 'relative_humidity': 38.6,
 'station': 948701,
 'windspeed_knots': 1.8}
{'GHI_w/m2': 48,
 '_id': ObjectId('664f0a66ab9e5e59a4469126'),
 'air_temperature_celcius': 5,
 'date': datetime.datetime(2023, 6, 30, 0, 0),
 'hotspots': [{'confidence': 78,
               'datetime': datetime.datetime(2023, 6, 30, 4, 41, 25),
               'latitude': -36.834,
     

#### Task 2-2 j 

In [94]:
result = collection.find({
    "precipitation": {
        "$regex": "^0\\.(2[0-9]|3[0-5])G$"
    }
})

# output
for document in result:
    pprint(document)


{'GHI_w/m2': 157,
 '_id': ObjectId('664f0a66ab9e5e59a446907e'),
 'air_temperature_celcius': 19,
 'date': datetime.datetime(2023, 1, 13, 0, 0),
 'hotspots': [],
 'max_wind_speed': 18.1,
 'precipitation': '0.31G',
 'relative_humidity': 54.1,
 'station': 948700,
 'windspeed_knots': 11.2}
{'GHI_w/m2': 146,
 '_id': ObjectId('664f0a66ab9e5e59a44690c9'),
 'air_temperature_celcius': 17,
 'date': datetime.datetime(2023, 3, 29, 0, 0),
 'hotspots': [{'confidence': 69,
               'datetime': datetime.datetime(2023, 3, 29, 0, 48, 40),
               'latitude': -34.2648,
               'longitude': 141.6325,
               'surface_temperature_celcius': 51}],
 'max_wind_speed': 21.0,
 'precipitation': '0.24G',
 'relative_humidity': 49.9,
 'station': 948701,
 'windspeed_knots': 12.2}
{'GHI_w/m2': 166,
 '_id': ObjectId('664f0a66ab9e5e59a44690df'),
 'air_temperature_celcius': 20,
 'date': datetime.datetime(2023, 4, 20, 0, 0),
 'hotspots': [{'confidence': 84,
               'datetime': datetime.dat

#### Task 2 - 3

In [95]:
collection.create_index([("date", 1), ("surface_temperature_celcius", 1)])

'date_1_surface_temperature_celcius_1'

I would choose attributes "date" and "surface_temperature" because it will optimise the mongodb. At first, I was considering about adding the "station", but I noticed that there is no "station" in the hotspot. Therefore, "date" and "surface_temperature" would be the best decision.

In [96]:
indexes = collection.index_information()

pprint(indexes)

{'_id_': {'key': [('_id', 1)], 'v': 2},
 'date_1_surface_temperature_celcius_1': {'key': [('date', 1),
                                                  ('surface_temperature_celcius',
                                                   1)],
                                          'v': 2}}
